This software is Copyright © 2024 The Regents of the University of California. All Rights Reserved. Permission to copy, modify, and distribute this software and its documentation for educational, research and non-profit purposes, without fee, and without a written agreement is hereby granted, provided that the above copyright notice, this paragraph and the following three paragraphs appear in all copies. Permission to make commercial use of this software may be obtained by contacting:

Office of Innovation and Commercialization

9500 Gilman Drive, Mail Code 0910

University of California

La Jolla, CA 92093-0910

(858) 534-5815

invent@ucsd.edu

This software program and documentation are copyrighted by The Regents of the University of California. The software program and documentation are supplied “as is”, without any accompanying services from The Regents. The Regents does not warrant that the operation of the program will be uninterrupted or error-free. The end-user understands that the program was developed for research purposes and is advised not to rely exclusively on the program for any reason.

IN NO EVENT SHALL THE UNIVERSITY OF CALIFORNIA BE LIABLE TO ANY PARTY FOR DIRECT, INDIRECT, SPECIAL, INCIDENTAL, OR CONSEQUENTIAL DAMAGES, INCLUDING LOST PROFITS, ARISING OUT OF THE USE OF THIS SOFTWARE AND ITS DOCUMENTATION, EVEN IF THE UNIVERSITY OF CALIFORNIA HAS BEEN ADVISED OF THE POSSIBILITY OF SUCH DAMAGE. THE UNIVERSITY OF CALIFORNIA SPECIFICALLY DISCLAIMS ANY WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE. THE SOFTWARE PROVIDED HEREUNDER IS ON AN “AS IS” BASIS, AND THE UNIVERSITY OF CALIFORNIA HAS NO OBLIGATIONS TO PROVIDE MAINTENANCE, SUPPORT, UPDATES, ENHANCEMENTS, OR MODIFICATIONS.

In [ ]:
import pandas as pd
import gzip
from collections import OrderedDict, defaultdict
import ipaddress
import math
import radix
import re
import numpy as np
import matplotlib.pyplot as plt
import bz2
from fuzzywuzzy import fuzz
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
split = re.compile(r'(.+?):\s*(.*)')

class Entry(OrderedDict):
    def __repr__(self):
        output = []
        for key, value in self.items():
            output.append('{}:\t{}'.format(key, value))
        return '\n'.join(output)
    
    @property
    def date(self):
        # self refers to the object, which in this case is a subclass of dict
        # the .get method for dict objects retrieves the value if the key exists,
        # otherwise it returns the default value
        changed = self.get('changed', None)
        if changed is not None:
            try:
                date = changed.split()[1]
            except IndexError:
                return '17000101'
        else:
            try:
                date = self['last-modified'].replace('-', '')
            except KeyError:
                return '16000101'
        return date
    
    
def parse_whois(filename):
    with gzip.open(filename, 'rt', encoding='latin-1') as f:
        items = []  # list to hold whois items
        item = Entry()  # an object to hold an individual entry

        # Iterate over the lines in the whois file
        for line in f:
            ol = line  # original version of the line
            line = line.strip()  # get rid of whitespace at beginning and end of the line

            # If the original line is not just a newline character,
            # and the line is only whitespace or starts with a comment character,
            # skip the line
            if ol != '\n' and (not line or line[0] == '#'):
                continue

            # If the line is not just whitespace
            if line:
                
                # if original line start with whitespace, append it to previous value
                if re.match(r'^[^a-zA-Z0-9]', ol):
                    try:
                        item[k] += '\n' + line
                        continue
                    except:
                        # print(item)
                        continue

                
                # See if the line matches the regex
                m = split.match(line)
                # If it does
                if m:
                    # There are 2 possible matching groups, so assign them to k and v
                    k, v = m.groups()
                    
                    # If the key is already in the item, concatenate the value to the existing value
                    if k in item and k != 'origin':
                        try:
                            item[k] += '\n' + v
                        except:
                            # print(item)
                            pass
                    # When the key does not yet exist in the item, add the key and value
                    else:
                        try:
                            item[k] = v
                        except:
                            # print(item)
                            pass
                # If it does not match
                else:
                    # Add the value to the previous key in the item
                    # This is a value with newline breaks
                    try:
                        item[k] += '\n' + line
                    except:
                        item[k] = line

            # If the line is just a newline break, finish the current item, and start a new one
            else:
                if item:
                    if set(['NetHandle', 'OrgName', 'ASHandle', 'Customer']) & set(item.keys()):
                        items.append(item)  # Add item to list of items
                    item = Entry()  # Start new item
    return items

In [ ]:
def build_as_rel():
    path = '' # Path to your AS relationship dataset
    as_rel = defaultdict(lambda: defaultdict(set))
    with bz2.open(path, 'rt') as file:
        for line in file:
            if line.startswith('#'):
                continue
            data = line.strip().split('|')
            AS1 = 'AS'+data[0]
            AS2 = 'AS'+data[1]
            rel = data[2]
            if rel == '0':
                as_rel[AS1]['peer'].add(AS2)
                as_rel[AS2]['peer'].add(AS1)
            elif rel == '1':
                as_rel[AS1]['provider'].add(AS2)
                as_rel[AS2]['customer'].add(AS1)
            elif rel == '-1':
                as_rel[AS1]['customer'].add(AS2)
                as_rel[AS2]['provider'].add(AS1)
    return as_rel


def check_asrel(left, right): # left has to be customer of right
    try:
        for i in left:
            for j in right:
                if i in as_rel[j]['customer']:
                    return True
        return False
    except:
        return False

def build_as2org():
    path = '' # Path to your AS Organizations dataset
    as2org = gzip.open(path)
    mapping = {}
    companyname = {}
    for line in as2org:
        l = line.decode().strip('\n').split('|')
        asn = None
        if l[0].isdigit():
            asn = 'AS'+str(l[0])
            mapping[asn] = l[3]
        elif len(l) == 5:
            companyname[l[0]] = l[2]
    for i in mapping:
        orgname = companyname[mapping[i]]
        mapping[i] = orgname
    return mapping

def check_as2org(left, right):
    try:
        for i in left:
            for j in right:
                if as2org[i] == as2org[j]:
                    return True
        return False
    except:
        return False

as_rel = build_as_rel()
as2org = build_as2org()

def to_org(asn):
    try:
        return as2org[asn]
    except:
        return None

# Load arin DB

In [ ]:
arin = parse_whois('') # Path to your ARIN WHOIS database file
len(arin)

In [ ]:
arinaddr = pd.DataFrame([i for i in arin if 'NetHandle' in i])

In [ ]:
arinorg = pd.DataFrame([i for i in arin if 'OrgName' in i])

In [ ]:
arinorg['OrgName'].replace('Amazon Technologies Inc.', 'Amazon.com, Inc.', inplace=True)

In [ ]:
arinas = pd.DataFrame([i for i in arin if 'ASHandle' in i])

In [ ]:
arinas = arinas[arinas.OrgID != 'IANA']

In [ ]:
def expand_asn(x):
    if '-' not in x:
        return [int(x)]
    start, end = x.split('-')
    start = int(start)
    end = int(end)
    return list(range(start, end+1))

In [ ]:
arinas['ASN'] = arinas['ASNumber'].apply(expand_asn)

In [ ]:
arinas = arinas.explode('ASN')

In [ ]:
arinas['ASN'] = arinas['ASN'].apply(lambda x: 'AS'+str(x))

In [ ]:
arin

In [ ]:
print(len(arinaddr), len(arinorg), len(arinas))

In [ ]:
arinaddr['NetType'].value_counts()

# Prep for inetnum tree construction

In [ ]:
def range_to_pfx(inetnum):
    start, end = inetnum.strip().split('-')
    start_ip = ipaddress.IPv4Address(start.strip())
    end_ip = ipaddress.IPv4Address(end.strip())
    if int(end_ip) - int(start_ip) + 1 < 256:
        return None
    cidr_blocks = ipaddress.summarize_address_range(start_ip, end_ip)
    res = []
    for i in cidr_blocks:
        if i.prefixlen > 24:
            return None
        res.append(str(i))
    return res

In [ ]:
arinaddr['prefix'] = arinaddr['NetRange'].apply(range_to_pfx)

In [ ]:
tree_candidate = arinaddr.dropna(subset=['prefix'])

In [ ]:
len(tree_candidate)

In [ ]:
len(arinaddr) - len(tree_candidate)

In [ ]:
tree_candidate['blocks_in_chunk'] = tree_candidate['prefix'].apply(len)

In [ ]:
tree = tree_candidate.explode('prefix')

In [ ]:
tree = tree[tree.NetType != 'rir']

In [ ]:
rtree = radix.Radix()
for i in tree.prefix:
    rtree.add(i)

In [ ]:
tree.prefix.count()

In [ ]:
tree.prefix.nunique()

In [ ]:
tree.NetType.value_counts()

# match allocated PA to ASN and orgs

In [ ]:
allocated = tree[tree.NetType == 'allocation']

In [ ]:
allocated = allocated.merge(arinorg[['OrgID', 'OrgName']], on='OrgID', how='left')

In [ ]:
arinasgroups = arinas[['ASN', 'OrgID']].merge(arinorg[['OrgName', 'OrgID']]).groupby('OrgName').agg({'ASN': list}).reset_index()

In [ ]:
allocated = allocated.merge(arinasgroups, how='left')

In [ ]:
allocated.rename(columns={'ASN':'aut-num'}, inplace=True)

# find leaf nodes, suballocated PA or assigned PA

In [ ]:
def is_leaf(prefix):
    res = rtree.search_covered(prefix)
    if len(res) == 1:
        return True
    return False

In [ ]:
tree['leaf'] = tree['prefix'].apply(is_leaf)

In [ ]:
tree['leaf'].value_counts()

In [ ]:
leafs = tree[(tree.leaf == True) & tree.NetType.isin(['reallocation', 'reassignment'])]

In [ ]:
len(leafs)

In [ ]:
def is_root(prefix):
    res = rtree.search_covering(prefix)
    res = [i.prefix for i in res if i.prefix != '0.0.0.0/0']
    if len(res) == 1:
        return True
    return False

In [ ]:
tree['is_root'] = tree['prefix'].apply(is_root)

In [ ]:
tree[tree.is_root == True]

# find root of leaf

In [ ]:
atree = radix.Radix()
for i in allocated.prefix:
    atree.add(i)

In [ ]:
def find_root(prefix):
    curr = rtree.search_exact(prefix)
    while curr.parent and curr.parent.prefix != '0.0.0.0/0':
        curr = curr.parent
    return curr.prefix

In [ ]:
leafs['root'] = leafs['prefix'].apply(find_root)

In [ ]:
leafs

# find BGP origin of leaf prefix

In [ ]:
bgptree = radix.Radix()
# Path to your prefix2as dataset
with gzip.open('PATH TO YOUR prefix2as DATASET', 'rt') as file:
    for line in file:
        data = line.strip().split()
        pfx = data[0]
        asns = []
        for i in data[1].split('_'):
            for j in i.split(','):
                asns.append('AS'+j)
        node = bgptree.add(pfx)
        node.data['asn'] = asns

In [ ]:
def find_exact_origin(prefix):
    rnode = bgptree.search_exact(prefix)
    if rnode:
        return rnode.data['asn']
    return None

In [ ]:
leafs['exact_origin'] = leafs['prefix'].apply(find_exact_origin)

In [ ]:
print(len(leafs[~pd.isnull(leafs.exact_origin)]), len(leafs))

In [ ]:
leafs['root_origin'] = leafs['root'].apply(find_exact_origin)

In [ ]:
leafs

# Group 3: Child in BGP, parent not in BGP

In [ ]:
c1 = leafs[(~pd.isnull(leafs.exact_origin)) & (pd.isnull(leafs.root_origin))]

In [ ]:
c1

In [ ]:
c1 = c1.merge(allocated[['prefix', 'OrgName', 'aut-num']].rename(columns={'prefix': 'root'}))

## check if parent assigned ASN is related to child BGP origin AS

In [ ]:
c1['sibling'] = c1.apply(lambda x: check_as2org(x['exact_origin'], x['aut-num']), axis=1)

In [ ]:
c1['cp'] = c1.apply(lambda x: check_asrel(x['exact_origin'], x['aut-num']), axis=1)

In [ ]:
c1

In [ ]:
c1infer = c1[(c1.sibling == False) & (c1.cp == False)]

In [ ]:
c1infer

# Group 4: Both in BGP, unrelated

In [ ]:
c2 = leafs[(~pd.isnull(leafs.exact_origin)) & (~pd.isnull(leafs.root_origin))]
c2 = c2.merge(allocated[['prefix', 'OrgName', 'aut-num']].rename(columns={'prefix': 'root'}))

In [ ]:
c2['aut_sibling'] = c2.apply(lambda x: check_as2org(x['exact_origin'], x['aut-num']), axis=1)
c2['aut_cp'] = c2.apply(lambda x: check_asrel(x['exact_origin'], x['aut-num']), axis=1)

In [ ]:
c2['bgp_sibling'] = c2.apply(lambda x: check_as2org(x['exact_origin'], x['root_origin']), axis=1)
c2['bgp_cp'] = c2.apply(lambda x: check_asrel(x['exact_origin'], x['root_origin']), axis=1)

In [ ]:
c2infer = c2[(c2.aut_sibling == False) & (c2.aut_cp == False) & (c2.bgp_sibling == False) & (c2.bgp_cp == False)]

In [ ]:
c2infer

# Group 1: Neither in BGP

In [ ]:
c3 = leafs[(pd.isnull(leafs.exact_origin)) & (pd.isnull(leafs.root_origin))]

In [ ]:
c3

# Group 2: child not in BGP, parent in BGP

In [ ]:
c4 = leafs[(pd.isnull(leafs.exact_origin)) & (~pd.isnull(leafs.root_origin))]

In [ ]:
c4

# All categories: stats describe

In [ ]:
print('Total:', len(leafs))
print('Group 1: Unused:', len(c3))
print('Group 2: Aggregated Customer:', len(c4))
print('Group 3: ISP customer', len(c1) - len(c1infer), 'leased prefixes:', len(c1infer))
print('Group 4: Delegated customer', len(c2) - len(c2infer), 'leased prefixes:', len(c2infer))

# Inferred leases

In [ ]:
arin_inferred_lease = pd.concat([c1infer, c2infer])

In [ ]:
leased_prefix = set(arin_inferred_lease['prefix'])

In [ ]:
arin_inferred_lease = arin_inferred_lease.merge(arinorg[['OrgID', 'Country']])

In [ ]:
arin_inferred_lease['Country'].value_counts()[:20]

In [ ]:
arin_inferred_lease = arin_inferred_lease.merge(allocated[['prefix', 'OrgID']], left_on=['root'], right_on=['prefix'], suffixes=('', '_root'))

In [ ]:
arin_inferred_lease = arin_inferred_lease.merge(arinorg[['OrgID', 'Country']], left_on='OrgID_root', right_on='OrgID', suffixes=('_leaf', '_root'))

In [ ]:
arin_inferred_lease

In [ ]:
arin_inferred_lease['out_of_country'] = arin_inferred_lease['Country_leaf'] != arin_inferred_lease['Country_root']

In [ ]:
arin_inferred_lease['out_of_country'].value_counts()

In [ ]:
arin_inferred_lease[arin_inferred_lease.out_of_country]['Country_leaf'].value_counts()[:20]

## Top IP Holders

In [ ]:
arin_inferred_lease['OrgName'].value_counts()[:10]

## Top Originator

In [ ]:
originator = pd.DataFrame(arin_inferred_lease['exact_origin'].explode().value_counts().reset_index())

In [ ]:
originator['org'] = originator['exact_origin'].apply(to_org)

In [ ]:
originator[:20]

# Reference dataset: ARIN brokers

In [ ]:
brokerdf = pd.read_csv('recognized_brokers_arin.txt', delimiter=';', names=['broker', 'OrgID'])

In [ ]:
brokerdf

In [ ]:
tp = leafs[leafs.OrgID.isin(brokerdf.OrgID)].dropna(subset=['exact_origin'])